In [1]:
import pandas as pd
import numpy as np
import matplotlib
import time
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [19]:
train = pd.read_csv("train_ready_for_model.csv")
train.head()

,Gender_F,Gender_M,Product_ID,Age,City_Category,Marital_Status,New_Product_ID,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Purchase,Stay_In_Current_City_Years,User_ID
0,1,0,P00069042,1,0,0,69042,10,3,999.0,999.0,8370.0,2,1000001
1,1,0,P00248942,1,0,0,248942,10,1,6.0,14.0,15200.0,2,1000001
2,1,0,P00087842,1,0,0,87842,10,12,999.0,999.0,1422.0,2,1000001
3,1,0,P00085442,1,0,0,85442,10,12,14.0,999.0,1057.0,2,1000001
4,0,1,P00285442,7,2,0,285442,16,8,999.0,999.0,7969.0,4,1000002


In [20]:
test = pd.read_csv("test_ready_for_model.csv")
test.head()

,Gender_F,Gender_M,Product_ID,Age,City_Category,Marital_Status,New_Product_ID,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Stay_In_Current_City_Years,User_ID
0,0,1,P00128942,5,1,1,128942,7,1,11.0,999.0,2,1000004
1,0,1,P00113442,3,2,0,113442,17,3,5.0,999.0,0,1000009
2,1,0,P00288442,4,1,1,288442,1,5,14.0,999.0,4,1000010
3,1,0,P00145342,4,1,1,145342,1,4,9.0,999.0,4,1000010
4,1,0,P00053842,3,2,0,53842,1,4,5.0,12.0,1,1000011


In [15]:
#import pandas_profiling
#pandas_profiling.ProfileReport(train)

Number of variables,14
Number of observations,550068
Total Missing (%),0.0%
Total size in memory,58.8 MiB
Average record size in memory,112.0 B
Numeric,10
Categorical,1
Boolean,3
Date,0
Text (Unique),0
Rejected,0


In [17]:
result = test[["User_ID","Product_ID"]]
result.head()

,User_ID,Product_ID
0,1000004,P00128942
1,1000009,P00113442
2,1000010,P00288442
3,1000010,P00145342
4,1000011,P00053842


In [18]:
#train["Product_ID"]=train["Product_ID"].apply(lambda x:x[1:])

In [8]:
#test["Product_ID"]=test["Product_ID"].apply(lambda x:x[1:])

In [11]:
#Define target and ID columns:
target = 'Purchase'
IDcol =[]# ['User_ID','Product_ID']
from sklearn import model_selection, metrics

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    t1=time.time()
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])

    #Perform cross-validation:
    #cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='neg_mean_squared_error')
    #cv_train_score = np.sqrt(np.abs(cv_train_score))
    
    #Print model report:
    print("\nModel Report")
    #print("Train CV score : ", cv_train_score)
    print("Train RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)
    t2=time.time()
    t_linalg_lstsq = float(t2-t1)
    print("Time taken: {} seconds".format(t_linalg_lstsq))

In [12]:
from sklearn.linear_model import LinearRegression
predictors = [x for x in train.columns if x not in [target]]
#print("predictors : ",predictors)
# print predictors
alg1 = LinearRegression(normalize=False)
modelfit(alg1, train, test, predictors, target, IDcol, 'res2.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#print(coef1)
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train RMSE : 4960
Time taken: 2.5798418521881104 seconds


In [9]:
from sklearn.linear_model import Ridge
predictors = [x for x in train.columns if x not in [target]+IDcol]
#print("predictors : ",predictors)
# print predictors
alg1 = Ridge(alpha=0.1, normalize=False)
#Ridge(alpha = 1.0,normalize=)
modelfit(alg1, train, test, predictors, target, IDcol, 'res3.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train RMSE : 4984
Time taken: 1.2817161083221436 seconds


In [27]:
from sklearn.linear_model import Lasso
#predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = Lasso(alpha=0.1, normalize=False)
#Lasso(alpha = 1)
modelfit(alg1, train, test, predictors, target, IDcol, 'res4.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train RMSE : 4344
Time taken: 133.4852385520935 seconds


In [34]:
from sklearn.linear_model import ElasticNet
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = ElasticNet(alpha=0.1,l1_ratio=0.8,normalize=False)
#ElasticNet(alpha=0.01,l1_ratio=0.5,normalize=True)
modelfit(alg1, train, test, predictors, target, IDcol, 'res5.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train RMSE : 4485
Time taken: 8.349120616912842 seconds


In [8]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
for i in range(1,20):
    alg1 = KNeighborsRegressor(n_neighbors=i)
    modelfit(alg1, train, test, predictors, target, IDcol, 'resKN'+str(i)+'.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

In [ ]:
from sklearn.ensemble import RandomForestRegressor
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = RandomForestRegressor(n_estimators=100,max_depth=7, min_samples_split=10,min_samples_leaf=5,n_jobs=-1)
#alg1 = RandomForestRegressor(n_estimators=80,max_depth=15)
#alg1 = RandomForestRegressor(n_estimators=500,max_depth=10, min_samples_split=100,min_samples_leaf=50)
modelfit(alg1, train, test, predictors, target, IDcol, 'res8.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')

In [10]:
from sklearn.ensemble import AdaBoostRegressor
#predictors = [x for x in train.columns if x not in [target]+IDcol]
#print predictors
alg1 = AdaBoostRegressor(n_estimators=20 )
modelfit(alg1, train, test, predictors, target, IDcol, 'res9.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train RMSE : 4911
Time taken: 124.43570327758789 seconds


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
#GradientBoostingRegressor(alpha=0.3,n_estimators=100)
alg1 = GradientBoostingRegressor()
modelfit(alg1, train, test, predictors, target, IDcol, 'res9.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')